In [5]:
import pandas as pd
import sqlite3

In [8]:
# Load the subset dataset

df = pd.read_csv('Data/creditcard_subset.csv')
print("Columns:", df.columns.tolist())
print("Shape:", df.shape)

Columns: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']
Shape: (50000, 31)


In [9]:
# Create SQLite database
conn = sqlite3.connect('paysim.db')
df.to_sql('transactions', conn, if_exists='replace', index=False)
print("Database created and data loaded.")

Database created and data loaded.


Fraud Count by Hour:



In [10]:
query_hour = """
SELECT (Time / 3600) AS Hour, COUNT(*) AS Fraud_Count
FROM transactions
WHERE Class = 1
GROUP BY Hour
ORDER BY Fraud_Count DESC
"""
fraud_by_hour = pd.read_sql(query_hour, conn)
fraud_by_hour.to_csv('fraud_by_hour.csv', index=False)
print(fraud_by_hour.head())

        Hour  Fraud_Count
0  47.041944            1
1  47.040833            1
2  44.765000            1
3  44.693056            1
4  44.453889            1


Average Amount by Fraud Status:



In [11]:
query_amount = """
SELECT Class, AVG(Amount) AS Avg_Amount
FROM transactions
GROUP BY Class
"""
amount_by_fraud = pd.read_sql(query_amount, conn)
amount_by_fraud.to_csv('amount_by_fraud.csv', index=False)
print(amount_by_fraud)

   Class  Avg_Amount
0      0   87.249921
1      1  164.233855


Top 10 Fraud Transactions by Amount:

In [ ]:
query_top = """
SELECT Time, Amount
From transactions
WHERE Class = 1
ORDER BY Amount DESC
LIMIT 10
"""
top_fraud = pd.read_sql(query_top, conn)
top_fraud.to_csv('top_fraud.csv', index=False)
print(top_fraud)